# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pypokerengine.players import BasePokerPlayer
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate
import inspect
from pypokerengine.engine.hand_evaluator import HandEvaluator
from pypokerengine.engine.card import Card
import importlib
from random import shuffle
import time

# Gemini Bot

In [ ]:
class AggressivePokerPlayer(BasePokerPlayer):

    def __init__(self):
        super().__init__() 
        self.name = "AggressiveBot"

    def declare_action(self, valid_actions, hole_card, round_state):
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(
            nb_simulation=100,
            nb_player=self.nb_player,
            hole_card=gen_cards(hole_card),
            community_card=gen_cards(community_card)
        )

        # Preflop strategy
        if round_state['street'] == 'preflop':
            if win_rate > 0.65:
                if valid_actions[2]['amount']['min'] != -1:
                    return 'raise', valid_actions[2]['amount']['max']
                else:
                    return 'call', valid_actions[1]['amount']
            elif win_rate > 0.5:
                if valid_actions[1]['amount'] != 0:
                    return 'call', valid_actions[1]['amount']
                else:
                    return 'fold', 0
            else:
                return 'fold', 0

        # Postflop strategy
        else:
            if win_rate > 0.7:
                if valid_actions[2]['amount']['min'] != -1:
                    return 'raise', int(valid_actions[2]['amount']['max'] * 0.75) #75% of max raise
                else:
                    return 'call', valid_actions[1]['amount']
            elif win_rate > 0.55:
                 if valid_actions[2]['amount']['min'] != -1:
                    return 'raise', valid_actions[2]['amount']['min']
                 else:
                    return 'call', valid_actions[1]['amount']
            elif valid_actions[1]['amount'] < round_state['small_blind_amount']*2: #if call amount is small relative to the blinds, call with some weaker hands
                return 'call', valid_actions[1]['amount']
            else:
                return 'fold', 0

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

# Honest Player

In [ ]:
NB_SIMULATION = 100
class HonestPlayer(BasePokerPlayer):
    def __init__(self, raise_wr = 0.6, call_wr = 0.3):
        super().__init__() 
        self.raise_wr = raise_wr
        self.call_wr = call_wr
        
    def declare_action(self, valid_actions, hole_card, round_state):
        curframe = inspect.currentframe()
        calframe = inspect.getouterframes(curframe, 2)
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(
                nb_simulation=NB_SIMULATION,
                nb_player=self.nb_player,
                hole_card=gen_cards(hole_card),
                community_card=gen_cards(community_card)
                )
        if win_rate >= self.raise_wr:
            action = valid_actions[2]  # fetch bet action info
            valid_amounts = action['amount']
            raise_amount =  (valid_amounts['max'] - valid_amounts['min'])/ 5.0 + valid_amounts['min']
            return valid_actions[2]['action'], raise_amount
        elif win_rate >= self.call_wr:
            action = valid_actions[1]  # fetch CALL action info
        else:
            call_amount = valid_actions[1]['amount']
            if(call_amount == 0):
                return valid_actions[1]['action'], valid_actions[1]['amount']
            action = valid_actions[0]  # fetch FOLD action info
        return action['action'], action['amount']

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

# Bij Player

In [ ]:
def iterate_actions(hand_data):
    """Iterates over all actions in the 'action_histories' of a given hand.
    
    Args:
    hand_data: A dictionary representing a hand of poker.
    
    Yields:
    A tuple containing the action type, the amount, the player's UUID, and the round.
    """
    if(hand_data):
        for round_name, actions in hand_data['action_histories'].items():
            for action in actions:
                yield (action['action'], action.get('amount'), action['uuid'], round_name)

class BijPlayer(BasePokerPlayer):
    def __init__(self):
        super().__init__() 
        self.player_histories = {}
        self.player_tendencies = {}
        self.hand_evaluator = HandEvaluator()
        self.INVERTED_HAND_STRENGTH_MAP = {value: key for key, value in self.hand_evaluator.HAND_STRENGTH_MAP.items()}
        
        self.n_samples = 10
        self.player_showdown_score_histories = {}
        self.player_showdown_score_metrics = {}

        self.player_states = []

    def make_score(self, ri):
        hand_strength = self.INVERTED_HAND_STRENGTH_MAP[ri['hand']['strength']]
        hand_high = ri['hand']['high']
        hand_low = ri['hand']['low']
        hole_high = ri['hole']['high']
        hole_low = ri['hole']['low']
        return (hand_strength << 8) | (hand_high << 12) | (hand_low << 8) | (hole_high << 4) | hole_low

        
    def declare_action(self, valid_actions, hole_card, round_state):    
        pot_size = round_state['pot']['main']['amount']
        community_cards = [Card.from_str(c) for c in round_state['community_card']]
        hole_cards = [Card.from_str(c) for c in hole_card]

        my_hand_rank_info = self.hand_evaluator.gen_hand_rank_info(hole_cards, community_cards)
        my_hand_score = self.make_score(my_hand_rank_info)


        participating_players = [player for player in round_state['seats'] if(player['state'] != 'folded')]

        # Determine how many their showdowns we can beat
        matchups = []
        call_amount = valid_actions[1]['amount']
        max_bet_amount = valid_actions[2]['amount']['max']
        candidate_bets = np.linspace(call_amount, max_bet_amount)
        # win/los = (player_idx, showdown_idx)
        # fold = (player_idx, bet_idx, showdown_idx)
        
        for opponent in participating_players:
            opponent_showdown_history = self.player_showdown_score_histories[opponent['uuid']]
            matchup_my_win = np.zeros((len(opponent_showdown_history)))
            matchup_they_fold = np.zeros((len(candidate_bets), len(opponent_showdown_history)))
            
            for showdown_idx, showdown in enumerate(opponent_showdown_history):
                if(my_hand_score >= showdown['score']):
                    matchup_my_win[showdown_idx] = 1
                else:
                    matchup_my_win[showdown_idx] = 0
                 
                for bet_idx, bet_size in enumerate(candidate_bets):
                    if((bet_size / showdown['pot']) > (showdown['pot'] / showdown['cost'])):
                         matchup_they_fold[bet_idx, showdown_idx] = 1
                    else:
                        matchup_they_fold[bet_idx, showdown_idx] = 0
                

            # Save matchups
            matchups += [(matchup_my_win, matchup_they_fold)]
        
        # For each bet size, we sample 10 matchups from each player and calculate their value. The avg is the expected value
        # float (bet_idx)
        bet_evs = np.zeros_like(candidate_bets)
        for bet_idx, bet_size in enumerate(candidate_bets):
            # (sample_idx)
            they_fold_samples = []
            my_win_samples = []
            for opponent_idx, (matchup_my_win, matchup_they_fold) in enumerate(matchups):
                if(len(matchup_they_fold[bet_idx]) < 1):
                    # We don't have hand history for this player: assume 50/50
                    they_fold_samples += [np.random.rand(self.n_samples) > 0.5]
                    my_win_samples += [np.random.rand(self.n_samples) > 0.8]
                else:
                    # sample 10 folds for each
                    sample_idxs = np.random.choice(len(matchup_they_fold[bet_idx]), size=self.n_samples, replace=True)
                    they_fold_samples += [matchup_they_fold[bet_idx][sample_idxs]]
                    my_win_samples += [matchup_my_win[sample_idxs]]

            # (player_idx, sample_idx)
            they_fold_samples = np.stack(they_fold_samples, axis=0)
            # (player_idx, sample_idx)
            my_win_samples = np.stack(my_win_samples, axis=0)
            # Decide whether I win the entire pot: I beat everyone
            # bool (sample_idx)
            my_win_pot = np.sum(my_win_samples, axis=0) > 0
            # For each scenario (sample_idx) calculate how much I win given the folds/calls
            # float (sample_idx) 
            win_amt = pot_size + bet_size * np.sum(they_fold_samples, axis=0)
            win_amt = win_amt * my_win_pot
            los_amt = bet_size
            result = win_amt * my_win_pot - los_amt * np.logical_not(my_win_pot)
            bet_evs[bet_idx] = np.mean(result)

        best_bet_idx = np.argmax(bet_evs)
        best_bet_size = candidate_bets[best_bet_idx]
        best_bet_ev = bet_evs[bet_idx]

        if(best_bet_ev < 0):
            return valid_actions[0]['action'], valid_actions[0]['amount']
        else:
            return valid_actions[2]['action'], best_bet_size

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']
        for seat in game_info['seats']:
            self.player_showdown_score_histories[seat['uuid']] = []
        

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        self.player_states += [round_state['seats'][round_state['next_player']]['state']]
        #pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        showdown_players_uuids = [player['uuid'] for player in round_state['seats'] if(player['state'] != 'folded')]
        folded_players_uuids = [player['uuid'] for player in round_state['seats'] if(player['state'] == 'folded')]

        players_showdown_states = {player['uuid']: player['state'] for player in round_state['seats']}

        # Get cost for each player
        player_costs = {}
        for street, player_actions in round_state['action_histories'].items():
            for action in player_actions:
                uuid = action['uuid']
                try:
                    amount = action['amount']
                except KeyError:
                    amount = 0
                try:
                    player_costs[uuid] += amount
                except KeyError:
                    player_costs[uuid] = amount

        # NOTE THIS DOES NOT INCLUDE SIDE POTS
        pot_size = round_state['pot']['main']['amount']
        
        # Record player showdown states (score, cost, pot)
        showdown_player_hands = {player['uuid']: player['hand'] for player in hand_info}
        
        for player_uuid, hand in showdown_player_hands.items():
            hand = showdown_player_hands[player_uuid]
            score = self.make_score(hand)


            cost = player_costs[player_uuid]

            showdown_state = {'score': score, 'pot': pot_size, 'cost': cost}
            try:
                self.player_showdown_score_histories[player_uuid] += [showdown_state]
            except KeyError:
                self.player_showdown_score_histories[player_uuid] = [showdown_state]

# Gabe and Bittner Player

In [ ]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate

import math#, ttictoc
import importlib
class Digby(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"

    def __init__(self):
        super().__init__()
        importlib.reload(math)
        #importlib.reload(ttictoc)
        

    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [Fold, call, raise]
        #ttictoc.tic()
        # GATHER data on amount invested, and potential earnings
        call_amount=valid_actions[1]["amount"] # amount needed to call
        pot=round_state['pot']['main']['amount'] # current value of the pot
        self.current_stack #current stack
       
        #---- HAND ANALYSIS ----
        hole_card = gen_cards(hole_card)
        community_card = gen_cards(self.cc)
        win_p=estimate_hole_card_win_rate(nb_simulation=round(1000/self.nb_players), nb_player=self.nb_players, hole_card=hole_card, community_card=community_card) #current win probability

        #---- DECISION ANALYSIS ----
        
        #play tighter as your stack goes down
        if self.current_stack < 1200:
            set_evt = .05
        elif self.current_stack < 700:
            set_evt = 3
        elif self.current_stack < 400:
            set_evt = 6
        else: 
            set_evt = 0
        
        #play more loose as the game nears end
        set_evt=set_evt*(1- round_state['round_count']/1000)
        
        raise_amount=0 #initial raise of 0
        stay_flag=0
        EV=(win_p*(pot+call_amount+raise_amount*self.nb_players))-((1-win_p)*(self.amount_invested+call_amount+raise_amount))
        self.ev.append(EV)
        # if postive EV and you can raise -> cheeck to see if raising impacts EV then raise
        if EV>set_evt and valid_actions[2]['amount']['min']<self.current_stack:
            stay_flag=1 #staying in the game
            if win_p*self.nb_players+win_p - 1 > 0: #if derivative of EV(raise_amount) is positive linearly increasing and worth it to raise      
                #raise_factor=max(math.floor(win_p*self.nb_players),1) #determine agressiveness of raise
                raise_factor=max(math.floor(EV),1) #determine agressiveness of raise
            else: #if derivative is negative - need to call
                raise_factor=0
            raise_amount=raise_factor*valid_actions[2]['amount']['min']
       
        # Decide action
        if call_amount>0 and stay_flag==0: #if negative EV and and more money is needed to stay in game - FOLD
            action=valid_actions[0]['action']
            amount=valid_actions[0]['amount']
        elif call_amount==0 and stay_flag==0: #if negative EV and and no money is needed to stay in game - CALL
            action =valid_actions[1]['action']
            amount=valid_actions[1]['amount']
        elif stay_flag==1 and raise_amount==0: #if positive EV but raise amount is 0 - CALL
            action =valid_actions[1]['action']
            amount = valid_actions[1]['amount']
        else:  #if positive EV and nonzero raise amount - RAISE
            action = valid_actions[2]['action']
            amount = min(raise_amount,valid_actions[2]['amount']['max'],self.current_stack)
       
       
        self.amount_invested = self.amount_invested + amount
        
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        self.ev=[]
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        self.amount_invested=0 #resets counter of current amount invested in round to 0
        self.current_stack=seats[0]['stack'] #grabs stak of first seat, fix later to check that player id matches to get correct stack
        self.nb_players=len([player for player in seats if player['state'] != 'folded']) #grabs number of current players participating in game
        self.cc=[]
        pass

    def receive_street_start_message(self, street, round_state):
        self.nb_players=len([player for player in round_state['seats'] if player['state'] != 'folded'])
        self.cc=round_state['community_card']
        pass

    def receive_game_update_message(self, action, round_state):
        self.nb_players=len([player for player in round_state['seats'] if player['state'] != 'folded'])
        self.cc=round_state['community_card']
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass    


In [ ]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.engine.card import Card
from pypokerengine.engine.hand_evaluator import HandEvaluator
import random

class CompetitivePokerBot(BasePokerPlayer):
    def __init__(self):
        super().__init__()
        
    def declare_action(self, valid_actions, hole_card, round_state):
        """
        Decide the action based on hand strength, game state, and opponents' actions.

        :param valid_actions: List of actions with their constraints.
        :param hole_card: List of the bot's two hole cards.
        :param round_state: The state of the current poker round.
        :return: A tuple of the chosen action and its amount (if applicable).
        """
        community_cards = round_state["community_card"]
        hand_strength = self._estimate_hand_strength(hole_card, community_cards)
        opponent_aggressiveness = self._evaluate_opponents(round_state)

        # Adjust thresholds based on opponents' actions
        aggressive_threshold = 0.8 - 0.2 * opponent_aggressiveness
        cautious_threshold = 0.4 - 0.1 * opponent_aggressiveness

        if hand_strength >= aggressive_threshold:
            # Raise if allowed and hand strength is high
            for action in valid_actions:
                if action["action"] == "raise":
                    amount = np.random.rand(1)[0]*(action["amount"]["max"] -action["amount"]["min"]) + action["amount"]["min"]
                    return action["action"], amount

        if hand_strength >= cautious_threshold:
            # Call if hand strength is moderate
            for action in valid_actions:
                if action["action"] == "call":
                    return action["action"], action["amount"]

        # Fold if hand strength is weak
        for action in valid_actions:
            if action["action"] == "fold":
                return action["action"], action["amount"]

        # Default action (call or fold as a fallback)
        return valid_actions[0]["action"], valid_actions[0]["amount"]

    def receive_game_start_message(self, game_info):
        """
        Handle the game start message.

        :param game_info: Details of the game configuration.
        """
        self.game_info = game_info

    def receive_round_start_message(self, round_count, hole_card, seats):
        """
        Handle the round start message.

        :param round_count: The current round number.
        :param hole_card: The bot's hole cards for this round.
        :param seats: Information about all players.
        """
        self.hole_card = hole_card

    def receive_street_start_message(self, street, round_state):
        """
        Handle the street start message.

        :param street: The current street (preflop, flop, turn, or river).
        :param round_state: The current round state.
        """
        pass
        
    def receive_game_update_message(self, action, round_state):
        """
        Handle the game update message.

        :param action: The action that just occurred.
        :param round_state: The current round state.
        """
        self._track_opponent_action(action)
        pass
        
    def receive_round_result_message(self, winners, hand_info, round_state):
        """
        Handle the round result message.

        :param winners: Information about the winners.
        :param hand_info: Details of hands shown at showdown (if any).
        :param round_state: The final state of the round.
        """
        pass
        
    def _estimate_hand_strength(self, hole_cards, community_cards):
        """
        Estimate the strength of the bot's hand using Monte Carlo simulations.

        :param hole_cards: The bot's two hole cards.
        :param community_cards: The community cards on the table.
        :return: A float representing hand strength (0.0 to 1.0).
        """
        num_simulations = 100
        wins = 0
        total = 0

        for _ in range(num_simulations):
            deck = [Card.from_id(i) for i in range(1, 53)]
            used_cards = [Card.from_str(card) for card in hole_cards + community_cards]
            deck = [card for card in deck if card not in used_cards]
            random.shuffle(deck)

            sim_community = community_cards + [str(card) for card in deck[:5 - len(community_cards)]]
            sim_opponent = [str(card) for card in deck[5 - len(community_cards):7 - len(community_cards)]]

            bot_score = HandEvaluator.eval_hand(
                [Card.from_str(hole_cards[0]), Card.from_str(hole_cards[1])],
                [Card.from_str(c) for c in sim_community]
            )
            opp_score = HandEvaluator.eval_hand(
                [Card.from_str(sim_opponent[0]), Card.from_str(sim_opponent[1])],
                [Card.from_str(c) for c in sim_community]
            )

            if bot_score > opp_score:
                wins += 1
            total += 1

        return wins / total

    def _track_opponent_action(self, action):
        """
        Track opponent actions to evaluate their aggressiveness.

        :param action: The action taken by an opponent.
        """
        if not hasattr(self, 'opponent_actions'):
            self.opponent_actions = []
        self.opponent_actions.append(action)

    def _evaluate_opponents(self, round_state):
        """
        Evaluate opponents' aggressiveness based on their actions.

        :param round_state: The current state of the round.
        :return: A float representing opponent aggressiveness (0.0 to 1.0).
        """
        if not hasattr(self, 'opponent_actions') or len(self.opponent_actions) == 0:
            return 0.0

        aggressive_actions = [
            action for action in self.opponent_actions
            if action["action"] in ["raise", "allin"]
        ]
        return len(aggressive_actions) / len(self.opponent_actions)


# Run Poker Game

In [ ]:
# results_dict = {}
# initial_stack = 1000
# max_hands_per_game = 1000
# hands_per_blind_level = 25
# num_total_games = 100

# blind_structure = {round_num: {'ante': 2**idx, 'small_blind': 2**(idx+1)} for idx, round_num in enumerate(range(0, max_hands_per_game, hands_per_blind_level))}
# print('blind_structure: ', blind_structure)
                    
# for i in range(num_total_games):
#     print('Starting game ', i)
#     names_and_bots = [('Bij', BijPlayer()), ('GeminiAgro', AggressivePokerPlayer()), ('GabeBittner', Digby()), ('ChatGPT', CompetitivePokerBot())]
#     shuffle(names_and_bots)
#     config = setup_config(max_round=max_hands_per_game, initial_stack=initial_stack, small_blind_amount=1)
#     config.set_blind_structure(blind_structure)
#     for name, player in names_and_bots:
#         config.register_player(name=name, algorithm=player)
    
#     game_result = start_poker(config, verbose=0)
#     players = game_result['players']
#     for player in players:
#         try:
#             results_dict[player['name']] += [player['stack'] - initial_stack]
#         except KeyError:
#             results_dict[player['name']] = [player['stack'] - initial_stack]

In [ ]:
from multiprocessing import Pool
from itertools import repeat
results_dict = {}
initial_stack = 1000
max_hands_per_game = 1000
hands_per_blind_level = 25
num_total_games = 1000
blind_structure = {round_num: {'ante': 2**idx, 'small_blind': 2**(idx+1)} for idx, round_num in enumerate(range(0, max_hands_per_game, hands_per_blind_level))}
print('blind_structure: ', blind_structure)
                  

def run_single_game(_):
    """Runs a single poker game with the given configuration."""
    config = setup_config(max_round=max_hands_per_game, initial_stack=initial_stack, small_blind_amount=1)
    config.set_blind_structure(blind_structure)
    names_and_bots = [
        ('Bij', BijPlayer()), 
         ('GeminiAgro', AggressivePokerPlayer()), 
         ('GabeBittner', Digby()), 
         ('ChatGPT', CompetitivePokerBot())]    
    shuffle(names_and_bots)
    for name, player in names_and_bots:
        config.register_player(name=name, algorithm=player)
    game_result = start_poker(config, verbose=0)
    return game_result

def parallelize_poker_games(num_total_games):
    """Parallelizes the poker game simulations."""
    with Pool() as pool:
        results = pool.map(run_single_game, repeat(None, num_total_games))

    results_dict = {}
    for game_result in results:
        players = game_result['players']
        for player in players:
            try:
                results_dict[player['name']] += [player['stack'] - initial_stack]
            except KeyError:
                results_dict[player['name']] = [player['stack'] - initial_stack]
    return results_dict


results_dict = parallelize_poker_games(num_total_games)
    

# Claude Bot (does not work yet)

In [ ]:
import random
import numpy as np
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import estimate_hole_card_win_rate

class AdvancedPokerBot(BasePokerPlayer):
    def __init__(self, name="AdvancedBot"):
        super().__init__()
        self.name = name
        
        # Memory and tracking attributes
        self.opponents = {}  # Track opponent behaviors
        self.hand_history = []  # Track our hand history
        self.current_position = None
        
        # Strategy parameters
        self.aggression_factor = 0.6  # Baseline aggression
        self.bluff_frequency = 0.15   # Base bluffing probability
    
    def declare_action(self, valid_actions, hole_cards, round_state):
        """
        Primary decision-making method for the poker bot.
        Combines multiple strategies to make intelligent decisions.
        """
        # Analyze current game state
        self.current_position = self._determine_position(round_state)
        my_hand_strength = self._evaluate_hand_strength(hole_cards)
        pot_odds = self._calculate_pot_odds(round_state)
        
        # Opponent behavior analysis
        opponent_tendencies = self._analyze_opponent_behaviors(round_state)
        
        # Calculate win probability
        win_rate = estimate_hole_card_win_rate(
            nb_simulation=1000,
            nb_player=len(round_state['seats']),
            hole_card=hole_cards
        )
        
        # Adaptive strategy selection
        if win_rate > 0.7:  # Strong hand
            return self._play_strong_hand(valid_actions, my_hand_strength, pot_odds)
        elif win_rate > 0.4:  # Moderate hand
            return self._play_moderate_hand(valid_actions, my_hand_strength, opponent_tendencies)
        else:  # Weak hand
            return self._play_weak_hand(valid_actions, opponent_tendencies)
    
    def _determine_position(self, round_state):
        """
        Determine the bot's position at the table.
        Early, Middle, or Late position affects strategy.
        """
        player_num = len(round_state['seats'])
        my_seat_index = next(
            i for i, seat in enumerate(round_state['seats']) 
            if seat['name'] == self.name
        )
        
        if my_seat_index < player_num // 3:
            return 'Early'
        elif my_seat_index < 2 * player_num // 3:
            return 'Middle'
        else:
            return 'Late'
    
    def _evaluate_hand_strength(self, hole_cards):
        """
        Comprehensive hand strength evaluation.
        Considers card ranks, potential for straights/flushes.
        """
        ranks = [card[1] for card in hole_cards]
        suits = [card[0] for card in hole_cards]
        
        # Pair detection
        if ranks[0] == ranks[1]:
            return 'Pair'
        
        # Suited cards
        if len(set(suits)) == 1:
            return 'Suited'
        
        # High card potential
        high_cards = ['10', 'J', 'Q', 'K', 'A']
        if any(card in high_cards for card in ranks):
            return 'High'
        
        return 'Weak'
    
    def _calculate_pot_odds(self, round_state):
        """
        Calculate current pot odds to inform betting decisions.
        """
        total_pot = round_state['pot']['main']['amount']
        current_bet = round_state['action_histories'][-1].get('amount', 0)
        return current_bet / (total_pot + current_bet) if total_pot > 0 else 0
    
    def _analyze_opponent_behaviors(self, round_state):
        """
        Dynamic opponent profiling and behavior analysis.
        """
        return {
            'aggression': random.uniform(0.3, 0.7),
            'bluff_likelihood': random.uniform(0.1, 0.3)
        }
    
    def _play_strong_hand(self, valid_actions, hand_strength, pot_odds):
        """
        Aggressive strategy for strong hands.
        """
        raise_options = [action for action in valid_actions if action['action'] == 'raise']
        call_options = [action for action in valid_actions if action['action'] == 'call']
        
        if raise_options:
            # Choose a substantial raise based on hand strength
            raise_amount = raise_options[0]['amount']['max']
            return self._make_action('raise', raise_amount)
        elif call_options:
            return self._make_action('call', call_options[0]['amount'])
        else:
            return self._make_action('call', 0)
    
    def _play_moderate_hand(self, valid_actions, hand_strength, opponent_tendencies):
        """
        Balanced strategy for moderate hands.
        """
        if random.random() < self.bluff_frequency * opponent_tendencies['bluff_likelihood']:
            return self._bluff(valid_actions)
        
        call_options = [action for action in valid_actions if action['action'] == 'call']
        if call_options:
            return self._make_action('call', call_options[0]['amount'])
        return self._make_action('fold', 0)
    
    def _play_weak_hand(self, valid_actions, opponent_tendencies):
        """
        Conservative strategy for weak hands.
        """
        if random.random() < self.bluff_frequency * opponent_tendencies['bluff_likelihood']:
            return self._bluff(valid_actions)
        
        fold_options = [action for action in valid_actions if action['action'] == 'fold']
        if fold_options:
            return self._make_action('fold', 0)
        return self._make_action('call', 0)
    
    def _bluff(self, valid_actions):
        """
        Intelligent bluffing mechanism.
        """
        raise_options = [action for action in valid_actions if action['action'] == 'raise']
        call_options = [action for action in valid_actions if action['action'] == 'call']
        
        if raise_options:
            bluff_raise = raise_options[0]['amount']['min']
            return self._make_action('raise', bluff_raise)
        elif call_options:
            return self._make_action('call', call_options[0]['amount'])
        return self._make_action('fold', 0)
    
    def _make_action(self, action, amount):
        """
        Standardized action execution.
        """
        return action, amount
    
    def receive_game_result(self, winners, prize):
        """
        Update bot's memory and learning based on game outcomes.
        """
        if self.name in [winner['name'] for winner in winners]:
            # Adjust strategy if successful
            self.aggression_factor *= 1.1
            self.bluff_frequency *= 1.05
    
    def receive_round_result(self, round_state, result_state):
        """
        Analyze round results to refine strategy.
        """
        self.hand_history.append(result_state)

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass


# Plot Results

In [ ]:
def plot_running_sums(data):
  """Plots the running sums of a dictionary of arrays.

  Args:
    data: A dictionary where the keys are labels and the values are arrays of numbers.
  """

  for label, values in data.items():
    running_sum = np.cumsum(values)
    plt.plot(running_sum, label=label)

  plt.xlabel("Index")
  plt.ylabel("Running Sum")
  plt.legend()
  plt.show()

plot_running_sums(results_dict)

In [ ]:
np.random.choice(20, size=10, replace=True)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

# Generate some sample data (replace with your own data source)
np.random.seed(0)  # for reproducibility
x_data = np.linspace(0, 10, 100)
y_data = np.sin(x_data) + np.random.normal(0, 0.5, 100)

# Create the figure and axes
fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)  # Initialize an empty line object

# Set axis limits (important for consistent animation frame)
ax.set_xlim(x_data.min(), x_data.max())
ax.set_ylim(y_data.min() - 1, y_data.max() + 1)  # Add some padding
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_title("Animated Data Plot")
ax.grid(True)


def animate(i):
    """Animation function that updates the plot in each frame."""
    x = x_data[:i+1]  # Get the first i+1 x values
    y = y_data[:i+1]  # Get the corresponding y values
    line.set_data(x, y)  # Update the line data
    return line,  # Return the updated line object (important for FuncAnimation)

# Create the animation
ani = animation.FuncAnimation(fig, animate, frames=len(x_data), interval=50, blit=True, repeat=False)
# frames: Number of frames (one for each data point)
# interval: Delay between frames in milliseconds (adjust for speed)
# blit: Optimizes drawing for smoother animation (if possible)
# repeat: Whether to loop the animation


plt.show()  # or ani.save('my_animation.gif', writer='pillow') to save as a gif

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML

# Generate sample data (same as before)
np.random.seed(0)
x_data = np.linspace(0, 10, 100)
y_data = np.sin(x_data) + np.random.normal(0, 0.5, 100)

fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)

ax.set_xlim(x_data.min(), x_data.max())
ax.set_ylim(y_data.min() - 1, y_data.max() + 1)
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_title("Animated Data Plot")
ax.grid(True)

def animate(i):
    x = x_data[:i+1]
    y = y_data[:i+1]
    line.set_data(x, y)
    return line,

ani = animation.FuncAnimation(fig, animate, frames=len(x_data), interval=50, blit=True, repeat=False)

# Crucial change for Jupyter: convert the animation to HTML5 video
HTML(ani.to_jshtml())

# No need for plt.show() in Jupyter when using HTML conversion